【問題1】公式Exampleを分担して実行

Image Segmentation with tf.keras

使用データはkaggleの中古車のセグメンテーション
中古車の画像は撮影された背景によって見にくくなってしまう
背景を別の色に塗りつぶして中古車を綺麗に見えるように自動化するのが目的

U-netを使うので最初にpaperを見る

ISBIセルトラッキングチャレンジ2015で勝利
このアーキテクチャーは、特徴を抽出する縮小パスと、
正確なローカライズを可能にする対称的な拡張パスから構成される
非常に少数の画像からend-to-endで訓練
高速であり、GPU環境では、512 x 512の画像のセグメンテーションに1秒かからない
IOUは93%を達成

収縮パスは畳み込みネットワークの典型的なアーキテクチャに従い２つの３×３畳み込みの繰り返し適用し、
ダウンサンプリングのための（ＲｅＬＵ）ストライド２および２×２マックスプーリングが続き
特徴チャンネルの数を２倍にする

拡張パスの各ステップはフィーチャマップのアップサンプリングとそれに続くフィーチャチャネル数を半分にする2×2の畳み込み縮小パスからの対応する切り取り済みフィーチャマップとの連結、および2つの3×3で構成される

クロッピングは、すべての畳み込みで境界ピクセルが失われるために必要
クラスにマッピングするために１×１畳み込みが使用

<img src="fig1.png" width="500" height="300">
<img src="fig2.png" width="300" height="300">
<img src="fig3.png" width="300" height="300">
<img src="fig4.png" width="300" height="300">

【問題2】Iris（2値分類）をKerasで学習

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.datasets import mnist
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを2値分類する
"""
# データセットの読み込み
dataset_path ="df_vergicolor_and_virginica.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
df = df[(df["target"] == "versicolor")|(df["target"] == "virginica")]
y = df["target"]
X = df.loc[:, ["sepal length (cm)", "sepal width (cm)", "petal length (cm)", "petal width (cm)"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
y[y=='versicolor'] = 0
y[y=='virginica'] = 1
y = y.astype(np.int)[:, np.newaxis]

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)


In [6]:
# model生成
input_data = tf.keras.layers.Input(shape=(X_train.shape[1],))
x = tf.keras.layers.Dense(50, activation=tf.nn.relu)(input_data)
x = tf.keras.layers.Dense(100, activation=tf.nn.relu)(x)
output = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)(x)
model = tf.keras.Model(inputs=input_data, outputs=output)

In [7]:
# compile
model.compile(loss='binary_crossentropy',
              optimizer=tf.train.AdamOptimizer(learning_rate=0.01),
              metrics=['accuracy'])

In [8]:
# fit
history = model.fit(X_train, y_train,
                    batch_size=10,
                    epochs=200,
                    verbose=1,
                    validation_data=(X_val, y_val))

Train on 64 samples, validate on 16 samples
Epoch 1/200
64/64 [==============================] - 2s 25ms/step - loss: 1.0392 - acc: 0.4375 - val_loss: 0.7082 - val_acc: 0.3750
Epoch 2/200
64/64 [==============================] - 0s 244us/step - loss: 0.5957 - acc: 0.6719 - val_loss: 0.5608 - val_acc: 0.7500
Epoch 3/200
64/64 [==============================] - 0s 488us/step - loss: 0.5288 - acc: 0.8281 - val_loss: 0.6581 - val_acc: 0.3750
Epoch 4/200
64/64 [==============================] - 0s 244us/step - loss: 0.5415 - acc: 0.6406 - val_loss: 0.4354 - val_acc: 0.8125
Epoch 5/200
64/64 [==============================] - 0s 244us/step - loss: 0.4042 - acc: 0.8594 - val_loss: 0.4280 - val_acc: 0.8125
Epoch 6/200
64/64 [==============================] - 0s 488us/step - loss: 0.3032 - acc: 0.9531 - val_loss: 0.3111 - val_acc: 0.8750
Epoch 7/200
64/64 [==============================] - 0s 423us/step - loss: 0.2443 - acc: 0.9531 - val_loss: 0.3744 - val_acc: 0.8125
Epoch 8/200
64/64 [=======

Epoch 62/200
64/64 [==============================] - 0s 343us/step - loss: 0.1682 - acc: 0.9375 - val_loss: 0.0749 - val_acc: 1.0000
Epoch 63/200
64/64 [==============================] - 0s 358us/step - loss: 0.1492 - acc: 0.9375 - val_loss: 0.0550 - val_acc: 1.0000
Epoch 64/200
64/64 [==============================] - 0s 343us/step - loss: 0.1129 - acc: 0.9375 - val_loss: 0.0457 - val_acc: 1.0000
Epoch 65/200
64/64 [==============================] - 0s 359us/step - loss: 0.1210 - acc: 0.9531 - val_loss: 0.0426 - val_acc: 1.0000
Epoch 66/200
64/64 [==============================] - 0s 358us/step - loss: 0.0716 - acc: 0.9687 - val_loss: 0.0865 - val_acc: 0.9375
Epoch 67/200
64/64 [==============================] - 0s 358us/step - loss: 0.0542 - acc: 0.9844 - val_loss: 0.0468 - val_acc: 1.0000
Epoch 68/200
64/64 [==============================] - 0s 343us/step - loss: 0.0669 - acc: 0.9531 - val_loss: 0.0941 - val_acc: 0.9375
Epoch 69/200
64/64 [==============================] - 0s 327us

Epoch 123/200
64/64 [==============================] - 0s 343us/step - loss: 0.0449 - acc: 0.9844 - val_loss: 0.0160 - val_acc: 1.0000
Epoch 124/200
64/64 [==============================] - 0s 343us/step - loss: 0.0589 - acc: 0.9687 - val_loss: 0.0628 - val_acc: 0.9375
Epoch 125/200
64/64 [==============================] - 0s 343us/step - loss: 0.0670 - acc: 0.9375 - val_loss: 0.0147 - val_acc: 1.0000
Epoch 126/200
64/64 [==============================] - 0s 374us/step - loss: 0.0454 - acc: 0.9844 - val_loss: 0.0333 - val_acc: 1.0000
Epoch 127/200
64/64 [==============================] - 0s 389us/step - loss: 0.0451 - acc: 0.9844 - val_loss: 0.0250 - val_acc: 1.0000
Epoch 128/200
64/64 [==============================] - 0s 374us/step - loss: 0.0778 - acc: 0.9688 - val_loss: 0.0371 - val_acc: 1.0000
Epoch 129/200
64/64 [==============================] - 0s 421us/step - loss: 0.0665 - acc: 0.9687 - val_loss: 0.0253 - val_acc: 1.0000
Epoch 130/200
64/64 [==============================] - 

Epoch 184/200
64/64 [==============================] - 0s 327us/step - loss: 0.0384 - acc: 0.9844 - val_loss: 0.0190 - val_acc: 1.0000
Epoch 185/200
64/64 [==============================] - 0s 327us/step - loss: 0.0323 - acc: 1.0000 - val_loss: 0.0264 - val_acc: 1.0000
Epoch 186/200
64/64 [==============================] - 0s 343us/step - loss: 0.0291 - acc: 1.0000 - val_loss: 0.0174 - val_acc: 1.0000
Epoch 187/200
64/64 [==============================] - 0s 327us/step - loss: 0.0366 - acc: 0.9844 - val_loss: 0.0845 - val_acc: 0.9375
Epoch 188/200
64/64 [==============================] - 0s 343us/step - loss: 0.0389 - acc: 0.9844 - val_loss: 0.0100 - val_acc: 1.0000
Epoch 189/200
64/64 [==============================] - 0s 327us/step - loss: 0.0496 - acc: 0.9844 - val_loss: 0.0239 - val_acc: 1.0000
Epoch 190/200
64/64 [==============================] - 0s 359us/step - loss: 0.0888 - acc: 0.9687 - val_loss: 0.0053 - val_acc: 1.0000
Epoch 191/200
64/64 [==============================] - 

In [9]:
# 確率の出力
y_pred_proba = model.predict(X_test)[:,0]

# predictの出力
y_pred = np.where(y_pred_proba>0.5, 1, 0)

print("y_pred_proba", y_pred_proba)
print("y_pred", y_pred)

y_pred_proba [5.2465210e-03 9.9998271e-01 2.9633627e-03 9.9999928e-01 9.9982506e-01
 9.9999321e-01 1.1255484e-01 9.9512744e-01 9.9999547e-01 9.9972719e-01
 9.9994397e-01 9.9988592e-01 9.9998987e-01 5.1823951e-02 6.4446082e-05
 3.2522026e-04 9.8249304e-01 1.5300071e-04 9.9887007e-01 5.6583958e-04]
y_pred [0 1 0 1 1 1 0 1 1 1 1 1 1 0 0 0 1 0 1 0]


In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 4)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 50)                250       
_________________________________________________________________
dense_4 (Dense)              (None, 100)               5100      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 101       
Total params: 5,451
Trainable params: 5,451
Non-trainable params: 0
_________________________________________________________________


【問題3】Iris（多値分類）をKerasで学習

In [11]:
dataset_path = "Iris.csv"
df = pd.read_csv(dataset_path)
X = df.iloc[:, 1:-1]
y = df.iloc[:, -1]
X = np.array(X)
y = np.array(y)
y[y=="Iris-setosa"] = 0
y[y=="Iris-versicolor"] = 1
y[y=="Iris-virginica"] = 2
y = y.astype(np.int)# ndarray(n_samples, n_features)
# onehotencoder
enc = OneHotEncoder(handle_unknown="ignore", sparse=False)
y_one_hot = enc.fit_transform(y[:, np.newaxis])

X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size = 0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [13]:
input_data = tf.keras.layers.Input(shape=(4,))
x = tf.keras.layers.Dense(50, activation=tf.nn.relu)(input_data)
x = tf.keras.layers.Dense(100, activation=tf.nn.relu)(x)
output = tf.keras.layers.Dense(3, activation=tf.nn.softmax)(x)
model = tf.keras.Model(inputs=input_data, outputs=output)

In [14]:
model.compile(loss='categorical_crossentropy',
              optimizer = tf.train.AdamOptimizer(learning_rate=0.01),
              metrics=['accuracy'])

In [15]:
history = model.fit(X_train, y_train,
          batch_size=10,
          epochs = 200,
          verbose = 1,
          validation_data=(X_val, y_val))

Train on 96 samples, validate on 24 samples
Epoch 1/200
96/96 [==============================] - 0s 3ms/step - loss: 1.0631 - acc: 0.5521 - val_loss: 0.5558 - val_acc: 0.9167
Epoch 2/200
96/96 [==============================] - 0s 520us/step - loss: 0.5128 - acc: 0.8021 - val_loss: 0.6173 - val_acc: 0.6250
Epoch 3/200
96/96 [==============================] - 0s 416us/step - loss: 0.3935 - acc: 0.7708 - val_loss: 0.3126 - val_acc: 0.9167
Epoch 4/200
96/96 [==============================] - 0s 415us/step - loss: 0.2542 - acc: 0.9271 - val_loss: 0.2553 - val_acc: 0.9583
Epoch 5/200
96/96 [==============================] - 0s 384us/step - loss: 0.1564 - acc: 0.9583 - val_loss: 0.2341 - val_acc: 0.8750
Epoch 6/200
96/96 [==============================] - 0s 395us/step - loss: 0.1413 - acc: 0.9583 - val_loss: 0.3756 - val_acc: 0.8333
Epoch 7/200
96/96 [==============================] - 0s 405us/step - loss: 0.1831 - acc: 0.9062 - val_loss: 0.3483 - val_acc: 0.8333
Epoch 8/200
96/96 [========

Epoch 62/200
96/96 [==============================] - 0s 353us/step - loss: 0.0730 - acc: 0.9583 - val_loss: 0.4862 - val_acc: 0.9167
Epoch 63/200
96/96 [==============================] - 0s 364us/step - loss: 0.1613 - acc: 0.9375 - val_loss: 0.2087 - val_acc: 0.8333
Epoch 64/200
96/96 [==============================] - 0s 322us/step - loss: 0.0477 - acc: 0.9896 - val_loss: 0.4031 - val_acc: 0.9167
Epoch 65/200
96/96 [==============================] - 0s 343us/step - loss: 0.0768 - acc: 0.9792 - val_loss: 0.1858 - val_acc: 0.9167
Epoch 66/200
96/96 [==============================] - 0s 333us/step - loss: 0.0585 - acc: 0.9792 - val_loss: 0.3605 - val_acc: 0.9167
Epoch 67/200
96/96 [==============================] - 0s 322us/step - loss: 0.1111 - acc: 0.9583 - val_loss: 0.2430 - val_acc: 0.8333
Epoch 68/200
96/96 [==============================] - 0s 332us/step - loss: 0.1248 - acc: 0.9375 - val_loss: 0.5606 - val_acc: 0.8333
Epoch 69/200
96/96 [==============================] - 0s 332us

Epoch 123/200
96/96 [==============================] - 0s 343us/step - loss: 0.0317 - acc: 0.9792 - val_loss: 0.2365 - val_acc: 0.9167
Epoch 124/200
96/96 [==============================] - 0s 332us/step - loss: 0.0259 - acc: 0.9896 - val_loss: 0.1873 - val_acc: 0.9167
Epoch 125/200
96/96 [==============================] - 0s 343us/step - loss: 0.0260 - acc: 0.9896 - val_loss: 0.3443 - val_acc: 0.9167
Epoch 126/200
96/96 [==============================] - 0s 332us/step - loss: 0.1196 - acc: 0.9583 - val_loss: 0.1794 - val_acc: 0.9583
Epoch 127/200
96/96 [==============================] - 0s 343us/step - loss: 0.1994 - acc: 0.9271 - val_loss: 0.4465 - val_acc: 0.9167
Epoch 128/200
96/96 [==============================] - 0s 332us/step - loss: 0.0581 - acc: 0.9687 - val_loss: 0.1756 - val_acc: 0.9167
Epoch 129/200
96/96 [==============================] - 0s 322us/step - loss: 0.0259 - acc: 1.0000 - val_loss: 0.2497 - val_acc: 0.9167
Epoch 130/200
96/96 [==============================] - 

Epoch 184/200
96/96 [==============================] - 0s 374us/step - loss: 0.0211 - acc: 0.9896 - val_loss: 0.1957 - val_acc: 0.9167
Epoch 185/200
96/96 [==============================] - 0s 405us/step - loss: 0.0312 - acc: 0.9896 - val_loss: 0.3461 - val_acc: 0.9167
Epoch 186/200
96/96 [==============================] - 0s 374us/step - loss: 0.0426 - acc: 0.9792 - val_loss: 0.1956 - val_acc: 0.9167
Epoch 187/200
96/96 [==============================] - 0s 364us/step - loss: 0.0426 - acc: 0.9896 - val_loss: 0.3729 - val_acc: 0.9167
Epoch 188/200
96/96 [==============================] - 0s 353us/step - loss: 0.0298 - acc: 0.9896 - val_loss: 0.2561 - val_acc: 0.9167
Epoch 189/200
96/96 [==============================] - 0s 353us/step - loss: 0.0211 - acc: 0.9896 - val_loss: 0.2796 - val_acc: 0.9167
Epoch 190/200
96/96 [==============================] - 0s 384us/step - loss: 0.0278 - acc: 0.9896 - val_loss: 0.3492 - val_acc: 0.9167
Epoch 191/200
96/96 [==============================] - 

【問題4】House Pricesのモデルを作成

In [18]:
dataset_path = "df_grlivearea_yearbuilt.csv"
df = pd.read_csv(dataset_path)
X = df.loc[:,['GrLivArea', 'YearBuilt']]
y = df.loc[:,'SalePrice']
X = np.array(X)
y = np.array(y)[:,np.newaxis]
scaler = StandardScaler()
scaler.fit_transform(X)
y_log = np.log(y)
X_train, X_test, y_train, y_test = train_test_split(X, y_log, test_size = 0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

C:\Users\umini\Anaconda3\envs\pytorch\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\umini\Anaconda3\envs\pytorch\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [20]:
input_data = tf.keras.layers.Input(shape=(X_train.shape[1],))
x = tf.keras.layers.Dense(50, activation=tf.nn.relu)(input_data)
x = tf.keras.layers.Dense(100, activation=tf.nn.relu)(x)
output = tf.keras.layers.Dense(1, activation=None)(x) # identity_function
model = tf.keras.Model(inputs=input_data, outputs=output)

In [23]:
# lossをmseに
model.compile(loss='mse',
              optimizer = tf.train.AdamOptimizer(learning_rate=0.01),
              metrics=['mse'])

In [24]:
model.fit(X_train, y_train,
          batch_size=50,
          epochs=50,
          verbose=1,
          validation_data=(X_val, y_val))

Train on 934 samples, validate on 234 samples
Epoch 1/50
934/934 [==============================] - 0s 333us/step - loss: 39506.6172 - mean_squared_error: 39506.6172 - val_loss: 3197.7952 - val_mean_squared_error: 3197.7952
Epoch 2/50
934/934 [==============================] - 0s 76us/step - loss: 2153.3951 - mean_squared_error: 2153.3951 - val_loss: 548.6060 - val_mean_squared_error: 548.6060
Epoch 3/50
934/934 [==============================] - 0s 57us/step - loss: 145.9982 - mean_squared_error: 145.9982 - val_loss: 2.5183 - val_mean_squared_error: 2.5183
Epoch 4/50
934/934 [==============================] - 0s 79us/step - loss: 22.7052 - mean_squared_error: 22.7052 - val_loss: 15.6023 - val_mean_squared_error: 15.6023
Epoch 5/50
934/934 [==============================] - 0s 68us/step - loss: 4.5116 - mean_squared_error: 4.5116 - val_loss: 0.3678 - val_mean_squared_error: 0.3678
Epoch 6/50
934/934 [==============================] - 0s 59us/step - loss: 0.8847 - mean_squared_error: 0.

【問題5】MNISTのモデルを作成

せっかくなのでconvolutionでやる

データ読み込み

In [64]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [65]:
X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255
print(X_train.max()) # 1.0
print(X_train.min()) # 0.0

1.0
0.0


In [66]:
X_train = X_train.reshape(-1, 28, 28, 1)

In [67]:
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train_one_hot = enc.fit_transform(y_train[:, np.newaxis])
y_test_one_hot = enc.transform(y_test[:, np.newaxis])
print(y_train.shape) # (60000,)
print(y_train_one_hot.shape) # (60000, 10)
print(y_train_one_hot.dtype) # float64
print(y_test_one_hot.shape)

(60000,)
(60000, 10)
float64
(10000, 10)


In [68]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train_one_hot, test_size=0.2)
print(X_train.shape) # (48000, 784)
print(X_val.shape) # (12000, 784)

(48000, 28, 28, 1)
(12000, 28, 28, 1)


model作成


In [69]:
# functional apiで2層のconvlutionと2層の全結合層
input_data = tf.keras.layers.Input(shape=(28, 28,1))
conv1 = tf.keras.layers.Conv2D(32, kernel_size=(3,3), activation=tf.nn.relu)(input_data)
pool1 = tf.keras.layers.MaxPool2D(pool_size=(2,2))(conv1)
conv2 = tf.keras.layers.Conv2D(64, kernel_size=(3,3), activation=tf.nn.relu)(pool1)
pool2 = tf.keras.layers.MaxPool2D(pool_size=(2,2))(conv2)
drop = tf.keras.layers.Dropout(rate=0.5)(pool2)
flat = tf.keras.layers.Flatten()(drop)
hidden1 = tf.keras.layers.Dense(128, activation=tf.nn.relu)(flat)
output = tf.keras.layers.Dense(10, activation=tf.nn.softmax)(hidden1)
model = tf.keras.Model(inputs=input_data, outputs=output)

In [70]:
model.compile(loss='categorical_crossentropy',
              optimizer=tf.train.AdamOptimizer(learning_rate=0.01),
              metrics=['accuracy'])

In [72]:
model.fit(X_train,y_train,
          batch_size=50,
          epochs=100,
          validation_data=(X_val, y_val))

Train on 48000 samples, validate on 12000 samples
Epoch 1/100
48000/48000 [==============================] - 5s 96us/step - loss: 0.1040 - acc: 0.9719 - val_loss: 0.0671 - val_acc: 0.9848
Epoch 2/100
48000/48000 [==============================] - 5s 99us/step - loss: 0.1059 - acc: 0.9716 - val_loss: 0.0745 - val_acc: 0.9810
Epoch 3/100
48000/48000 [==============================] - 5s 99us/step - loss: 0.1030 - acc: 0.9727 - val_loss: 0.0645 - val_acc: 0.9828
Epoch 4/100
48000/48000 [==============================] - 5s 100us/step - loss: 0.1041 - acc: 0.9722 - val_loss: 0.0708 - val_acc: 0.9816
Epoch 5/100
48000/48000 [==============================] - 5s 99us/step - loss: 0.1099 - acc: 0.9714 - val_loss: 0.0627 - val_acc: 0.9828
Epoch 6/100
48000/48000 [==============================] - 5s 99us/step - loss: 0.1094 - acc: 0.9730 - val_loss: 0.0598 - val_acc: 0.9847
Epoch 7/100
48000/48000 [==============================] - 5s 99us/step - loss: 0.1032 - acc: 0.9733 - val_loss: 0.0664 -